### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/brittneycontreras/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221130_scenario_projections.ipynb")

2022/11/30 12:37:03 INFO|============================================================


### Fit an initial scenario through present day

In [ ]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-09-14'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/11/30 14:26:56 INFO|-------------------------|{} Retrieving vaccinations data
2022/11/30 14:26:59 INFO|-------------------------|{} Constructing vaccination projections
2022/11/30 14:27:02 INFO|-------------------------|{} Retrieving hospitalizations data
2022/11/30 14:27:04 INFO|----------------------|{} Prepping Model
2022/11/30 14:27:04 INFO|-----------------------|{} Prepping Model
2022/11/30 14:29:15 INFO|----------------------|{} Model prepped for fitting in 130.67892237500018 seconds.
2022/11/30 14:29:15 INFO|----------------------|{} Will fit 32 times
2022/11/30 14:37:31 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/32 completed in 496.0925280829997 seconds: {0: {'co': 0.830636683881486}, 14: {'co': 0.6982129327797195}, 28: {'co': 0.569547286270425}, 42: {'co': 0.0027652678914775507}, 56: {'co': 0.6814719979284506}, 70: {'co': 0.8912751286725015}}
2022/11/30 14:37:31 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/11/3

### Create and run scenarios from Feb 2022 to present

In [ ]:
multiprocess = 4

scenario_params = json.load(open('covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json'))

model_args = {
    'base_spec_id': model.spec_id, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-09-15', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for (weak_escape, strong_escape) in [(0.6, 0.15), (0.66, 0.165), (0.72, 0.18), (0.54, 0.135)]:
    weak_param = [{"param": "immune_escape",
                   "from_attrs": {"immun": "weak",
                                  "variant": ["none", "wildtype", "alpha", "delta",
                                              "omicron", "ba2", "ba2121", "ba45"]},
                   "to_attrs": {"variant": ["emv"]},
                   "vals": {"2020-01-01": weak_escape},
                   "desc": "emerging variants immune escape value, weak immunity"}]
    strong_param = [{"param": "immune_escape",
                   "from_attrs": {"immun": "strong",
                                  "variant": ["none", "wildtype", "alpha", "delta",
                                              "omicron", "ba2", "ba2121", "ba45"]},
                   "to_attrs": {"variant": ["emv"]},
                   "vals": {"2020-01-01": strong_escape},
                   "desc": "emerging variants immune escape value, strong immunity"}]
    lt5_vacc_adjust = [{"param": "immunity",
                        "attrs": {'age': '0-19', 'vacc': 'shot1'},
                        "mults": {"2020-01-01": 1,
                                  "2022-06-24": 0.99 + 0.01*vacc_eff_lt5,
                                  "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                  "2022-07-08": 0.97 + 0.03*vacc_eff_lt5,
                                  "2022-07-19": 0.96 + 0.04*vacc_eff_lt5,
                                  "2022-07-29": 0.95 + 0.05*vacc_eff_lt5,
                                  "2022-08-11": 0.94 + 0.06*vacc_eff_lt5,
                                  "2022-08-30": 0.93 + 0.07*vacc_eff_lt5,
                                  "2022-09-26": 0.92 + 0.08*vacc_eff_lt5,
                                  "2022-10-26": 0.91 + 0.09*vacc_eff_lt5,},
                        "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
    scenario_model_args.append({'params_defs': scenario_params + weak_param + strong_param + lt5_vacc_adjust,
                                'tags': {'emv_escape_weak': weak_escape,
                                        'emv_escape_strong': strong_escape}})
            

In [10]:
# check how many scenarios there are
len(scenario_model_args)

4

In [11]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/11/30 14:10:09 INFO|============================================================
2022/11/30 14:10:09 INFO|============================================================
2022/11/30 14:10:09 INFO|============================================================
2022/11/30 14:10:09 INFO|============================================================
2022/11/30 14:10:16 INFO|-----------|{'emv_escape_weak': 0.66, 'emv_escape_strong': 0.165} Retrieving vaccinations data
2022/11/30 14:10:16 INFO|-----------|{'emv_escape_weak': 0.54, 'emv_escape_strong': 0.135} Retrieving vaccinations data
2022/11/30 14:10:16 INFO|-----------|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.15} Retrieving vaccinations data
2022/11/30 14:10:17 INFO|-----------|{'emv_escape_weak': 0.72, 'emv_escape_strong': 0.18} Retrieving vaccinations data
2022/11/30 14:10:19 INFO|-----------|{'emv_escape_weak': 0.66, 'emv_escape_strong': 0.165} Constructing vaccination projections
2022/11/30 14:10:19 INFO|-----------|{'emv_escape_w

Unexpected exception formatting exception. Falling back to standard exception


2022/11/30 14:13:26 INFO|--------|{'emv_escape_weak': 0.54, 'emv_escape_strong': 0.135} Model solved in 33.167691624999975 seconds.
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/Users/brittneycontreras/PycharmProjects/covid-models/covid_model/runnable_functions.py", line 251, in do_single_fit_wrapper_parallel
    return do_single_fit(**args, write_results=False)
  File "/Users/brittneycontreras/PycharmProjects/covid-models/covid_model/runnable_functions.py", line 200, in do_single_fit
    batch_tstarts =  relevant_tc_ts[:last_batch_start_index:tc_batch_increment] + [relevant_tc_ts[last_batch_start_index]]
IndexError: list index out of range
"""



### Run the report for each fit model

In [8]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45', 'emv'], from_date='2022-01-01')

NameError: name 'models' is not defined

In [7]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/11/30 14:06:34 INFO|Projecting


NameError: name 'models' is not defined

In [ ]:
model.solution_ydf.to_csv('covid_model/output/20221109_scenario_projections.ipynb/state_model_solution_ydf.csv', index=False)